# Empirical Asset Pricing - PS5

Maximilian Huber

## Task 1


In [1]:
using CSV, DataFrames, Query, Plots, Optim; gr();

INFO: Recompiling stale cache file C:\Users\Max\AppData\Local\JuliaPro-0.6.2.1\pkgs-0.6.2.1\lib\v0.6\QueryOperators.ji for module QueryOperators.


### (a)

In [53]:
data = CSV.read("./Data/PS5.csv", delim='\t')
data[:, 1] = Date.(data[:, 1], "mm/dd/yyyy")
data[:yearmonth] = Dates.yearmonth.((data[:, 1]))
data = by(data, :yearmonth) do df
    return df[indmax(Dates.day.(df[:Date])), 2:end-1] 
end;

### (b)

In [82]:
data = hcat(data, 
    DataFrame(
        hcat([n * data[:, 1+n] - (n - 1) * data[:, n] for n in 2:10]...), 
        Symbol.(["fwd$(y)y" for y in 2:10])));

In [83]:
head(data)

,yearmonth,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10,fwd2y,fwd3y,fwd4y,fwd5y,fwd6y,fwd7y,fwd8y,fwd9y,fwd10y
1,"(2018, 2)",2.0493,2.2631,2.4284,2.5559,2.6541,2.7297,2.7881,2.8336,2.8694,2.898,2.4769,2.759,2.9384,3.0469,3.1077,3.1385,3.1521,3.1558,3.1554
2,"(2018, 1)",1.9119,2.1441,2.3127,2.4354,2.525,2.5909,2.6398,2.6766,2.705,2.7275,2.3763,2.6499,2.8035,2.8834,2.9204,2.9332,2.9342,2.9322,2.93
3,"(2017, 12)",1.7573,1.8852,2.0033,2.1056,2.1912,2.2613,2.318,2.3638,2.4012,2.432,2.0131,2.2395,2.4125,2.5336,2.6118,2.6582,2.6844,2.7004,2.7092
4,"(2017, 11)",1.6313,1.7902,1.9286,2.0468,2.1465,2.23,2.3,2.3589,2.4088,2.4518,1.9491,2.2054,2.4014,2.5453,2.6475,2.72,2.7712,2.808,2.8388
5,"(2017, 10)",1.4416,1.5965,1.7537,1.897,2.0211,2.1259,2.2133,2.2864,2.3481,2.4009,1.7514,2.0681,2.3269,2.5175,2.6499,2.7377,2.7981,2.8417,2.8761
6,"(2017, 9)",1.3341,1.481,1.6455,1.8015,1.939,2.0561,2.1541,2.236,2.305,2.3638,1.6279,1.9745,2.2695,2.489,2.6416,2.7421,2.8093,2.857,2.893


### (c)
$$rx_{t+1} = p_{t+1}(n-1) - p_t(n) - y_t(1)$$

In [100]:
pt = hcat([- n * data[:, 1+n] for n in 1:10]...)
rx_t1 = pt[1:end-1, 1:9] - pt[2:end, 2:10] - repmat(data[2:end, 2], 1, 9);

In [103]:
(rx_t1)

555×9 Array{Float64,2}:
 0.327    0.5      0.5445   0.4895  …   0.1885  -0.0158  -0.2357  -0.4615
 0.1012  -0.0356  -0.273   -0.5429     -1.0767  -1.3255  -1.5593  -1.7823
 0.1918   0.3841   0.546    0.6788      0.9009   1.0139   1.1375   1.2759
 0.1201   0.2391   0.3606   0.4767      0.6715   0.7496   0.8201   0.8882
 0.1863   0.4094   0.6108   0.7729      0.9892   1.0608   1.1197   1.171 
 0.0793   0.1753   0.2161   0.1913  …   0.0007  -0.1285  -0.2607  -0.3872
 0.2431   0.6935   1.1842   1.6474      2.4372   2.7738   3.0858   3.3792
 0.2901   0.7413   1.1478   1.4512      1.7646   1.8143   1.8189   1.7897
 0.1185   0.3892   0.6248   0.7708      0.8416   0.8265   0.8129   0.818 
 0.2488   0.7568   1.2218   1.5887      2.099    2.2888   2.4542   2.6051
 0.3844   1.0104   1.6034   2.0749  …   2.6684   2.8472   2.9825   3.0958
 0.5019   1.0853   1.4779   1.6867      1.8046   1.8057   1.7996   1.796 
 0.6074   1.1429   1.5681   1.9034      2.4159   2.631    2.8323   3.0224
 ⋮            

In [148]:
rxbar_t1 = 1/9 * sum(rx_t1, 2)
X = hcat(ones(length(rxbar_t1)), data[2:end, :SVENY01], [data[2:end, Symbol("fwd$(y)y")] for y in 2:5]...)

555×6 Array{Float64,2}:
 1.0  1.9119  2.3763  2.6499  2.8035  2.8834
 1.0  1.7573  2.0131  2.2395  2.4125  2.5336
 1.0  1.6313  1.9491  2.2054  2.4014  2.5453
 1.0  1.4416  1.7514  2.0681  2.3269  2.5175
 1.0  1.3341  1.6279  1.9745  2.2695  2.489 
 1.0  1.2322  1.4134  1.7239  2.0153  2.2447
 1.0  1.2383  1.4753  1.8638  2.2146  2.4785
 1.0  1.2608  1.5284  1.9265  2.2703  2.518 
 1.0  1.1887  1.3793  1.7991  2.1621  2.4163
 1.0  1.1191  1.4375  1.8873  2.2641  2.529 
 1.0  1.0551  1.5035  2.0635  2.4803  2.7356
 1.0  0.9164  1.557   2.0869  2.4561  2.6891
 1.0  0.8732  1.5238  2.0925  2.5121  2.7914
 ⋮                                    ⋮     
 1.0  5.4437  6.2401  6.3762  6.3992  6.4033
 1.0  5.4438  6.1952  6.4045  6.4633  6.4792
 1.0  5.3263  6.1211  6.396   6.491   6.5241
 1.0  4.7773  6.0205  6.2731  6.3243  6.3343
 1.0  5.0769  5.9409  6.1926  6.266   6.2876
 1.0  4.569   5.5524  6.0396  6.2814  6.4011
 1.0  4.5459  5.8073  6.2251  6.3637  6.4095
 1.0  5.0208  6.1222  6.3014  6

In [149]:
γ = (X' * X) \ (X' * rxbar_t1)

6×1 Array{Float64,2}:
  -0.370198
  -1.941   
   5.4919  
 -10.7566  
   7.98636 
  -0.739936

In [150]:
CP = (X * γ)[:, 1]

555-element Array{Float64,1}:
 0.721465
 0.577565
 0.740145
 0.925075
 1.02497 
 0.890991
 1.13293 
 1.12205 
 1.02468 
 1.26188 
 1.42704 
 1.5795  
 1.79233 
 ⋮       
 1.1155  
 1.02007 
 1.12062 
 1.76485 
 1.18103 
 1.71803 
 1.81846 
 1.5975  
 2.14449 
 2.12131 
 1.79862 
 1.45012 

$$CP_t = \hat{\overline{rx}}_{t+1}$$

### (e)

In [151]:
X = hcat(ones(length(CP)), CP)
f⟂ = hcat([data[2:end, Symbol("fwd$(n)y")] - X * ((X' * X) \ (X' * data[2:end, Symbol("fwd$(n)y")])) for n in 2:10]...);

### (f)

In [144]:
decomp[2][:, end-1:end] ./ sum(decomp[2][:, end-1:end], 1)

9×2 Array{Float64,2}:
 -2.07346   0.123497
 -1.96663   0.121598
 -1.31468   0.117695
 -0.507959  0.113584
  0.279181  0.109841
  0.96282   0.10667 
  1.51044   0.104121
  1.91729   0.102183
  2.19301   0.100811

In [152]:
decomp = eig(cov(f⟂))
PC = f⟂ * decomp[2][:, end-1:end] ./ sum(decomp[2][:, end-1:end], 1)

555×2 Array{Float64,2}:
 -6.40323  -4.17463  
 -5.39532  -4.56306  
 -4.96229  -4.4493   
 -4.49614  -4.3967   
 -4.16745  -4.37884  
 -3.6547   -4.66447  
 -3.43469  -4.33696  
 -4.05077  -4.33279  
 -3.38242  -4.47523  
 -3.81175  -4.25409  
 -4.49519  -4.03086  
 -5.2773   -3.97295  
 -4.82464  -3.7669   
  ⋮                  
 -5.55606  -0.402889 
 -4.81071  -0.394712 
 -4.61024  -0.317302 
 -7.08932  -0.166146 
 -5.27216  -0.51479  
 -4.34138  -0.190327 
 -6.15009  -0.109487 
 -6.8893   -0.234283 
 -5.24443   0.0693883
 -5.75692   0.0740625
 -3.38269  -0.315897 
 -3.63671  -0.390235 

### (g)

In [158]:
CP = CP - mean(CP)
PC = PC .- mean(PC, 1)

555×2 Array{Float64,2}:
 -6.40323  -4.17463  
 -5.39532  -4.56306  
 -4.96229  -4.4493   
 -4.49614  -4.3967   
 -4.16745  -4.37884  
 -3.6547   -4.66447  
 -3.43469  -4.33696  
 -4.05077  -4.33279  
 -3.38242  -4.47523  
 -3.81175  -4.25409  
 -4.49519  -4.03086  
 -5.2773   -3.97295  
 -4.82464  -3.7669   
  ⋮                  
 -5.55606  -0.402889 
 -4.81071  -0.394712 
 -4.61024  -0.317302 
 -7.08932  -0.166146 
 -5.27216  -0.51479  
 -4.34138  -0.190327 
 -6.15009  -0.109487 
 -6.8893   -0.234283 
 -5.24443   0.0693883
 -5.75692   0.0740625
 -3.38269  -0.315897 
 -3.63671  -0.390235 

### (h)
#### (i)

In [163]:
X = hcat(ones(length(rxbar_t1)), data[2:end, :SVENY01], [data[2:end, Symbol("fwd$(y)y")] for y in 2:5]...)
γ = (X' * X) \ (X' * rxbar_t1)
1 - sum((rxbar_t1 - (X * γ)[:, 1]).^2)/sum((rxbar_t1 - mean(rxbar_t1)).^2)

0.43734213071923

#### (ii)
Until 1992.12 the $R^2$ is:

In [181]:
X = hcat(ones(length(rxbar_t1[218:end])), data[219:end, :SVENY01], [data[219:end, Symbol("fwd$(y)y")] for y in 2:5]...)
γ = (X' * X) \ (X' * rxbar_t1[218:end])
1 - sum((rxbar_t1[218:end] - (X * γ)[:, 1]).^2)/sum((rxbar_t1[218:end] - mean(rxbar_t1[218:end])).^2)

0.3581029321663768

From 1993 on the $R^2$ is:

In [183]:
X = hcat(ones(length(rxbar_t1[1:217])), data[2:218, :SVENY01], [data[2:218, Symbol("fwd$(y)y")] for y in 2:5]...)
γ = (X' * X) \ (X' * rxbar_t1[1:217])
1 - sum((rxbar_t1[1:217] - (X * γ)[:, 1]).^2)/sum((rxbar_t1[1:217] - mean(rxbar_t1[1:217])).^2)

0.5602113742763926

TODO: Interpret??

## Task 2

In [326]:
X = hcat(CP, PC)

555×3 Array{Float64,2}:
 -0.732341    -6.40323  -4.17463  
 -0.876241    -5.39532  -4.56306  
 -0.713662    -4.96229  -4.4493   
 -0.528732    -4.49614  -4.3967   
 -0.428833    -4.16745  -4.37884  
 -0.562815    -3.6547   -4.66447  
 -0.320873    -3.43469  -4.33696  
 -0.331753    -4.05077  -4.33279  
 -0.429128    -3.38242  -4.47523  
 -0.191922    -3.81175  -4.25409  
 -0.0267684   -4.49519  -4.03086  
  0.125697    -5.2773   -3.97295  
  0.338521    -4.82464  -3.7669   
  ⋮                               
 -0.338307    -5.55606  -0.402889 
 -0.433735    -4.81071  -0.394712 
 -0.333187    -4.61024  -0.317302 
  0.31104     -7.08932  -0.166146 
 -0.272781    -5.27216  -0.51479  
  0.264221    -4.34138  -0.190327 
  0.364653    -6.15009  -0.109487 
  0.143692    -6.8893   -0.234283 
  0.690681    -5.24443   0.0693883
  0.667508    -5.75692   0.0740625
  0.344815    -3.38269  -0.315897 
 -0.00368774  -3.63671  -0.390235 

In [327]:
y1 = data[2:end, :SVENY01]

555-element Array{Float64,1}:
 1.9119
 1.7573
 1.6313
 1.4416
 1.3341
 1.2322
 1.2383
 1.2608
 1.1887
 1.1191
 1.0551
 0.9164
 0.8732
 ⋮     
 5.4437
 5.4438
 5.3263
 4.7773
 5.0769
 4.569 
 4.5459
 5.0208
 4.2036
 4.243 
 4.3542
 4.6539

In [318]:
Z = hcat(ones(size(X, 1)), X)
δ = (Z' * Z) \ (Z' * y1)

4-element Array{Float64,1}:
  5.22048  
 -1.39617  
 -0.0083798
  1.04331  

In [319]:
1 - sum((y1 - (Z * δ)[:, 1]).^2)/sum((y1 - mean(y1)).^2)

0.9939130606952067

I guess it's correct?!

## Task 4

In [320]:
β = ((X[13:end, :])' * X[13:end, :]) \ ((X[13:end, :])' * X[1:end-12, :])

3×3 Array{Float64,2}:
  0.540006    0.0659785   -0.492155 
  0.0888033   0.422987     0.0906404
 -0.0061813  -0.00301876   0.900161 

naive OLS estimate:

In [343]:
ϵ = ((X[1:end-12, :])' .- β * (X[13:end, :])')'
Σhat = ϵ'ϵ/(size(ϵ, 1) - 9 - 1)
round.(kron(inv(X'X), Σhat), 3)

9×9 Array{Float64,2}:
  0.003  -0.001   0.001  -0.0     0.0    -0.0   0.0    -0.0     0.0
 -0.001   0.007  -0.0     0.0    -0.0     0.0  -0.0     0.0    -0.0
  0.001  -0.0     0.002  -0.0     0.0    -0.0   0.0    -0.0     0.0
 -0.0     0.0    -0.0     0.001  -0.0     0.0   0.0    -0.0     0.0
  0.0    -0.0     0.0    -0.0     0.001  -0.0  -0.0     0.0    -0.0
 -0.0     0.0    -0.0     0.0    -0.0     0.0   0.0    -0.0     0.0
  0.0    -0.0     0.0     0.0    -0.0     0.0   0.001  -0.0     0.0
 -0.0     0.0    -0.0    -0.0     0.0    -0.0  -0.0     0.002  -0.0
  0.0    -0.0     0.0     0.0    -0.0     0.0   0.0    -0.0     0.0

HAC version to control for autocorrelation:

In [341]:
function Shat_HAC(Y, X, β; Jn = 4)
    N = length(β)
    
    Γ = zeros(N, N)
    Γ!(Γ, Y, X, β, j = 0)
    
    #0th lag
    Shat = copy(Γ)
    
    #all other lags
    for j in 1:Jn
        Γ .= zeros(N, N)
        Γ!(Γ, Y, X, β, j = j)
        Shat .+= (1 - j / (Jn + 1)) .* (Γ .+ Γ')
    end
    
    return Shat
end

function Γ!(Γ, Y, X, β; j = 0)
    for t in j+1:size(X, 1)
        Γ .+= vec((Y[t, :] - β  * X[t, :]) * X[t, :]') * vec((Y[t, :] - β  * X[t, :]) * X[t, :]')'
    end
    
    Γ .= Γ ./ (size(X, 1) - j)
end

Γ! (generic function with 2 methods)

In [344]:
round.(Shat_HAC(X[1:end-12, :], X[13:end, :], β, Jn = 0)/size(X, 1), 3)

9×9 Array{Float64,2}:
  0.013  -0.008   0.005  -0.011   0.013  -0.005  -0.005   0.009  -0.006
 -0.008   0.054  -0.004   0.013  -0.028   0.007   0.009  -0.018  -0.001
  0.005  -0.004   0.014  -0.005   0.007  -0.014  -0.006  -0.001  -0.003
 -0.011   0.013  -0.005   0.072  -0.047   0.036   0.007  -0.016   0.004
  0.013  -0.028   0.007  -0.047   0.188  -0.027  -0.016   0.022   0.002
 -0.005   0.007  -0.014   0.036  -0.027   0.065   0.004   0.002   0.002
 -0.005   0.009  -0.006   0.007  -0.016   0.004   0.099  -0.04   -0.005
  0.009  -0.018  -0.001  -0.016   0.022   0.002  -0.04    0.107   0.011
 -0.006  -0.001  -0.003   0.004   0.002   0.002  -0.005   0.011   0.032